In [ ]:
def generateCommonNetwork(case_data, control_data, genename, lambda1=0.20, lambda2=0.1, threshold=1e-6):
    # feature size (gene size)
    p = control_data.shape[1]

    # sample size
    n1 = control_data.shape[0]
    n2 = case_data.shape[0]

    # start calculations
    diffedges = {}
    beta1_all = []
    beta2_all = []
    for gene in tqdm(range(p)):
        # choose one gene as target
        y = concatenateGeneData(control_data[:, gene], case_data[:, gene], method='row')

        # choose other genes as feature
        idx = [i for i in range(p) if i != gene]
        X = concatenateGeneData(control_data[:, idx], case_data[:, idx], method='diag')

        # perform bcd algorithm
        beta = bcd(y, X, lambda1, lambda2, n1, n2, threshold)

        # reindex the features
        beta1 = list(beta[0 : gene]) + [0] + list(beta[gene : p - 1])
        #beta1 = np.array(beta1)
        beta1_all.append(beta1)
        beta2 = list(beta[p - 1 : gene + p - 1]) + [0] + list(beta[gene + p - 1 : 2 * p - 2])
        #beta2 = np.array(beta2)
        beta2_all.append(beta2)
            
        # construct neighbours under two conditions
        condition1 = [genename[i] for i in range(p) if beta1[i] != 0 and beta2[i] == 0]
        condition2 = [genename[i] for i in range(p) if beta2[i] != 0 and beta1[i] == 0]
        weight1 = [beta1[i] for i in range(p) if beta1[i] != 0 and beta2[i] == 0]
        weight2 = [beta2[i] for i in range(p) if beta2[i] != 0 and beta1[i] == 0]
        

        

        # update results
        for neighbors, weights, condition in zip([condition1, condition2], [weight1, weight2], ['condition1', 'condition2']):
            for neighbor, weight in zip(neighbors, weights):
                tuple_diffedge = (min(genename[gene], neighbor), max(genename[gene], neighbor), condition)
                diffedges.setdefault(tuple_diffedge, 0.0)
                diffedges[tuple_diffedge] += weight

    diffedges = sorted([k + tuple([v]) for k, v in diffedges.items()])

    #return diffedges
    return beta1_all,beta2_all

In [ ]:
beta1_all,beta2_all = generateBasicNetwork(case_standard, control_standard, genename)
beta1_all = np.array(beta1_all)
beta2_all = np.array(beta2_all)

In [ ]:
# original network 1
H1 = nx.from_numpy_matrix(beta1_all)
mapping={0:'A',1:'B',2:'C',3:'D',4:'E',5:'F'} 
H1 = nx.relabel_nodes(H1,mapping) 
Diff = nx.difference(H1, H2)
pos = nx.circular_layout(Diff)
nx.draw(H1, pos,with_labels = True,node_color="tab:orange")
plt.title('original network 1')
plt.show()


In [ ]:
# original network 2
H2 = nx.from_numpy_matrix(beta2_all)
mapping={0:'A',1:'B',2:'C',3:'D',4:'E',5:'F'} 
H2 = nx.relabel_nodes(H2,mapping) 
pos = nx.circular_layout(H2)
nx.draw(H2, pos,with_labels = True,node_color="tab:green")
plt.title('original network 2')
plt.show()

In [ ]:
# common parts between two network
Common = nx.intersection(H1, H2)
pos = nx.circular_layout(Diff)
nx.draw(Common, pos,with_labels = True,node_color="tab:orange")
plt.title('common parts between two network')
plt.show()
